# Analysis of Live Coronavirus Data
## Introduction and motivation
Finding myself with an overabundance of time, I set myself to creating a few graphs that visualize the data being generated as a product of this current Coronavirus (CV) mess. You can find them [here](http://johnsherrill.heliohost.org/corona_graphs.html). One such graph was a plot of CV cases overtime by state:

In [86]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import datetime as dt
import statsmodels.api as sm

state_data = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv')

In [150]:
fig1 = px.line(state_data, x="date", y="cases", color='state', log_y=True,
               title='Figure 1: CV cases (note logarithmic y-axis) over time by state', 
               category_orders={'state': np.sort(state_data.state.unique()).tolist()},
               range_x=['2020-03-01', datetime.datetime.now()])
fig1.update(layout=dict(title=dict(x=0.5)))
fig1.show()

I started ruminating about possible explanations for differences in the rate of increases in reported CV cases. One obvious hypothesis I began to entertain was that population density is positively associated with rates of change in reported CV cases. 

Population density varies a lot within each state. The Anchorage metro area contains about 56% of the population of Alaska, the Las Vegas metro area contains about 72% of the population of Nevada, and (depending on how you define a metro area) New York City is actually larger in population that New York State. So investigating this hypothesis at the state level is... improper. Fortunately, the New York Times is maintaining a public dataset of reported CV cases at the **_county_** level. Available [here](https://github.com/nytimes/covid-19-data).

## Preliminary data wrangling
First, I calculated the population density in persons per sq mile for each county (this was surprisingly hard as census.gov has recently (as of April 3, 2020) changed up their website and a literal day of googling couldn't find me a table/csv/whatever of population density per county). 

Second, I manually recorded from a NYTimes [page](https://www.nytimes.com/interactive/2020/us/coronavirus-stay-at-home-order.html) the date each state had enacted a "shelter-in-place" edict. If a state had yet to do so, the current date will be used. 

Third, I calculated the data each state had surpassed 10 cases. The [code](https://github.com/joncheryl/corona-graphs/blob/master/static_data.py) for this process is omitted here because the data is static. We're just gonna read it in.

In [51]:
county_data = pd.read_csv('https://raw.githubusercontent.com/joncheryl/corona-graphs/master/county_data.csv',
                          dtype={'fips': 'string'},
                          parse_dates=['date_shutdown', 'date_surpass'])

Next, I selected from the NYTimes data referenced above, the counties that had reported at least two days with more than 10 cases. For each such county, I ran a simple linear regression of the log of the number of cases over time. I considered this slope an estimation of a county-specific ''reproduction number'', $R_0^{county}$ (see Appendix for future ideas).

In [55]:
c_days = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/'
                     'master/us-counties.csv',
                     dtype={'fips': 'string'},
                     parse_dates=['date'])

###
# Deal with 7 exceptions (NYC, KC, AS, GU, MP, PR, VI)
###
c_days.loc[c_days.county == 'Kansas City', 'fips'] = '29999'
c_days.loc[c_days.county == 'New York City', 'fips'] = '36999'
c_days.loc[c_days.county == 'American Samoa', 'fips'] = '60999'
c_days.loc[c_days.county == 'Guam', 'fips'] = '66999'
c_days.loc[c_days.county == 'Northern Mariana Islands', 'fips'] = '69999'
c_days.loc[c_days.county == 'Puerto Rico', 'fips'] = '72999'
c_days.loc[c_days.county == 'Virgin Islands', 'fips'] = '78999'

# remove days that have <= N cases
N = 10
c_days = c_days[c_days.cases > N]

# Find log_cases
c_days['log_cases'] = np.log(c_days.cases)

###
# Run a regression for each county
###

counties = c_days.fips.unique()

slope_data = []

for c in counties:
    temp = c_days.fips == c
    tempsum = temp.sum()

    # check if there's at least two data points to regress
    if tempsum > 1:
        X = c_days.date[temp]
        Y = c_days.log_cases[temp]

        # get dates in a usable format
        X = X.apply(lambda x: x.toordinal())
        X1 = sm.add_constant(X)
        model = sm.OLS(Y, X1).fit()

        slope_data.append([c, model.params[1], tempsum])

county_slopes = pd.DataFrame(slope_data, columns=['fips',
                                                  'slope',
                                                  'days_over'])

Lastly, I calculated the number of days between a state surpassing 10 CV cases and enacting a "shelter-in-place"/shutdown order. Then I put all the data together.

In [59]:
county_data = county_data.merge(county_slopes)

for i in county_data.index:
    if (pd.isnull(county_data.loc[i, 'date_shutdown'])):
        county_data.loc[i, 'date_shutdown'] = dt.datetime.today()

shut_int = county_data.date_shutdown.apply(lambda x: x.toordinal())
sur_int = county_data.date_surpass.apply(lambda x: x.toordinal())
time_diff = shut_int - sur_int
county_data['days_shutdown'] = time_diff

## Exploratory findings
### Slope vs log(density)
The first graphic generated was a scatter plot of the county regression slopes ($R_0^{county}$) vs the log of the population density of each county. Counties surpass 10 cases at different times and I think the counties that have more days past this point carry more signifigance. To add visual signifigance, the counties that have more days with over 10 cases are more opaque. The function I used for opacity was somewhat arbitrary. Your mileage may vary. If $O(d)$ represents the opacity corresponding to $d$ days then,

$$ O(d) = \sqrt{2d - d^2} .$$

In [77]:
county_data['log_density'] = np.log(county_data.density)

opac = county_data.days_over.copy()
opac = opac - opac.min() + .05
opac /= max(opac)
opac = 2*opac - opac**2
opac = opac.pow(.5)

fig2 = go.Figure(data=go.Scatter(x=county_data.log_density,
                                 y=county_data.slope,
                                 
                                 mode='markers',
                                 marker=dict(opacity=opac),
                                 text=county_data.days_over))
fig2.update_layout(
    title="Figure 2: Slope vs log(density) by county",
    title_x=.5,
    xaxis_title="log(Density)",
    yaxis_title="Slope"
)
fig2.show()

To my eyes, it seems like there is a real positive relationship between these two variables even though it doesn't explain much of the variability of Slope. I ran a regular OLS regression then a weighted regression. For weights, I used the opacity as defined above.

In [153]:
# Model 1: slope ~ density
X = county_data.log_density
Y = county_data.slope
X2 = sm.add_constant(X)
est_pre = sm.OLS(Y, X2)
est = est_pre.fit()
print(est.summary())
print()

# Model 1a: slope ~ density #### with weight on days in data per county
est_wls_pre = sm.WLS(Y, X2, opac)
est_wls = est_wls_pre.fit()
print(est_wls.summary())

                            OLS Regression Results                            
Dep. Variable:                  slope   R-squared:                       0.136
Model:                            OLS   Adj. R-squared:                  0.135
Method:                 Least Squares   F-statistic:                     136.9
Date:                Sun, 05 Apr 2020   Prob (F-statistic):           1.78e-29
Time:                        00:23:53   Log-Likelihood:                 1064.8
No. Observations:                 872   AIC:                            -2126.
Df Residuals:                     870   BIC:                            -2116.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.0500      0.010      5.058      

This confirms what was already seen in the scatter plot. And now we have an estimate for the relationship. 

The data changes everyday but today (April 4, 2020) the estimate is 0.0202 which means for a unit increase in log(density) the model estimates a 0.0202 increase in the slope. To getting a feeling for what this means, I'm going to compare two different counties and what the model estimates the CV case growth would be. First, Cache County, Utah has a population density of 96 people per sq mile. The model estimates a slope of $0.0548 + 0.0202 * log(96) = 0.1469$. Second, Salt Lake County has a population density of 1300 people per sq mile and the model estimates a slope of $0.0548 + 0.0202 * log(1300) = 0.1996$. A little bit of math and we get
Cache county: 

$$ cases(t) = e^{.1469 t} $$

Salt Lake County:

$$ cases(t) = e^{.1996 t} $$


We can graph these functions and can see they diverge pretty quick.

In [208]:
compare = pd.DataFrame({'days': list(range(0,30))})
Y_cache = np.exp(.1469 * compare)
compare['cases_per_1000'] = Y_cache/112
compare['county'] = 'Cache County'

compare2 = pd.DataFrame({'days': list(range(0,30))})
Y_sl = np.exp(.1996 * (compare2+7))
compare2['cases_per_1000'] = Y_sl/1127
compare2['county'] = 'Salt Lake County'
compare = compare.append(compare2, ignore_index=True)

figtest = px.line(compare, x="days", y="cases_per_1000", color='county',
               title='Figure example: tale of two counties')
figtest.update(layout=dict(title=dict(x=0.5)))
figtest.show()

Over the course of thirty days, Salt Lake County is estimated to have significantly more cases. This is what seems to be happening. On March 18, 2020 Cache County reported 2 cases and April 3, 2020 (16 days later) there were 15 cases. Thats' .118 per 1000 people.  On March 11, 2020 Salt Lake County reported 2 cases and 16 days later (March 27, 2020) there were 212. Thats .184 per 1000 people.

### Days under a shelter-in-place order
One would hope that shelter-in-place orders would have some effect on the public and reduce transmission rates. But shelter-in-place orders are not strictly enforced and these were recorded at the state level. I can't find this data at the county level and "shelter-in-place" may be too ill-defined for such data to be of value anyway. But regardless we can make the plot at the state level:

In [140]:
fig3 = go.Figure(data=go.Scatter(x=county_data.days_shutdown,
                                 y=county_data.slope,
                                 mode='markers',
                                 marker=dict(opacity=opac),
                                 text=county_data.days_over))
fig3.update_layout(
    title="Figure 3: Slope vs days under shelter-in-place by county",
    title_x=.5,
    xaxis_title="days state has sheltered-in-place",
    yaxis_title="Slope"
)
fig3.show()

Doesn't really look like there's much of a relationship. When we model it's effect, this is confirmed. Weighted or unweighted, the parameter estimate for the number of days in shutdown (seen in the "coef" columns below for "days_shutdown") is just barely below zero.

In [92]:
# Model 2: slope ~ density + shutdown_days
X1 = county_data[['log_density', 'days_shutdown']]
Y = county_data.slope
X = sm.add_constant(X1)
est2_pre = sm.OLS(Y, X)
est2 = est2_pre.fit()
print(est2.summary())
print()

# Model 2a: slope ~ density + shutdown_days ##### weight on days in data per county
est2_wls_pre = sm.WLS(Y, X, opac)
est2_wls = est2_wls_pre.fit()
print(est2_wls.summary())

                            OLS Regression Results                            
Dep. Variable:                  slope   R-squared:                       0.141
Model:                            OLS   Adj. R-squared:                  0.139
Method:                 Least Squares   F-statistic:                     71.09
Date:                Sat, 04 Apr 2020   Prob (F-statistic):           2.55e-29
Time:                        22:57:40   Log-Likelihood:                 1067.2
No. Observations:                 872   AIC:                            -2128.
Df Residuals:                     869   BIC:                            -2114.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.0627      0.011      5.463

## Mixed effect for state
Naturally, one would suspect that infection rates in each county would be related to rates in other counties in the same state. To investigate this, I grouped counties together by state. In these graphics, the counties appear on the x-axis according to its state's [FIPS code](https://en.wikipedia.org/wiki/Federal_Information_Processing_Standard_state_code).

In [158]:
# Scatter of slope for each state (fips encoded) with colors by log_density
fig4a = go.Figure(data=go.Scatter(x=county_data.state,
                                 y=county_data.slope,
                                 mode='markers',
                                 marker=dict(opacity=opac),
                                 text=county_data.days_over))
fig4a.update_layout(
    title="Figure 4a: Slopes grouped by state",
    title_x=.5,
    xaxis_title="State (FIPS code)",
    yaxis_title="Slope"
)
fig4a.show()

# box plots of slope by state
fig4b = px.box(county_data, x='state', y='slope', title='Figure 4b: for box-and-whisker fans')
fig4b.update(layout=dict(title=dict(x=0.5)))
fig4b.show()

The graphics do confirm variation between states. We can model this by including dummy variables for each state:

In [165]:
# Model 3: slope ~ density + state_group
X1 = county_data[['log_density', 'state']]
Y = county_data.slope
X = sm.add_constant(pd.get_dummies(X1, columns=['state']))
est3_pre = sm.OLS(Y, X)
est3 = est3_pre.fit()
#print(est3.summary())
#print()

# Model 3a: slope ~ density + state_group    ###### weighted
X1 = county_data[['log_density', 'state']]
Y = county_data.slope
X = sm.add_constant(pd.get_dummies(X1, columns=['state']))
est3_wls_pre = sm.WLS(Y, X, weights=opac)
est3_wls = est3_wls_pre.fit()
#print(est3_wls.summary())

The coefficient estimates aren't really that important and that's why I commented-out the summary tables. But there's a jump in $R^2$. Shouldn't be too surprising since a ton of little predictors were just thrown in. Comparing the weighted model above with just the log(density) predictor and this one we see a difference in $R^2$s of

In [160]:
est3_wls.rsquared - est_wls.rsquared

0.22765851229302736

But we can model the variance between states considering state as a random effect and using a mixed model. This is better for some reasons that are not clear to me.

In [148]:
model = sm.MixedLM.from_formula('slope ~ log_density',
                                county_data,
                                groups=county_data.state)
result = model.fit()
print(result.summary())

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: slope    
No. Observations: 872     Method:             REML     
No. Groups:       51      Scale:              0.0043   
Min. group size:  1       Log-Likelihood:     1101.0376
Max. group size:  53      Converged:          Yes      
Mean group size:  17.1                                 
-------------------------------------------------------
              Coef. Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------
Intercept     0.044    0.011  4.066 0.000  0.023  0.066
log_density   0.021    0.002 11.561 0.000  0.018  0.025
Group Var     0.001    0.004                           



/home/john/.local/lib/python3.6/site-packages/statsmodels/regression/mixed_linear_model.py:2149: ConvergenceWarning:

The MLE may be on the boundary of the parameter space.



And we can plot group estimates just for fun

In [161]:
RE_est = pd.DataFrame.from_dict(result.random_effects, orient='index')
RE_est = RE_est + result.params.Intercept

pre = {'state': county_data.state, 'fit': result.fittedvalues}
group_est = pd.DataFrame(data=pre)
group_est = group_est.groupby('state').mean()

figME = go.Figure(data=go.Scatter(x=county_data.state,
                                  y=county_data.slope,
                                  mode='markers',
                                  marker=dict(opacity=opac),
                                  text=county_data.days_over))

figME.add_trace(go.Scatter(x=group_est.index,
                           y=group_est.fit,
                           mode='markers',
                           marker=dict(size=8, color='oldlace', symbol='diamond-wide',
                                       line=dict(width=2, color='mediumvioletred'))))
figME.update_layout(
    title="Figure 5: with group estimates plotted",
    title_x=.5,
    xaxis_title="State (FIPS code)",
    yaxis_title="Slope"
)

figME.show()

## Concluding thoughts
It's nice that we found that there's a real relationship between the infection rate and population density. It's modest but real, as best as I can tell. 

But adding state as an explanatory variable or as a random effect doesn't seem very helpful. Yes, $R^2$ went up and perhaps the predictive abilities of the model increase but states aren't random/arbitrary. We want to know **_why_** the states are different. Including them as a random effect or whatever doesn't get us any closer. I think we want to know what variable at the state level explains this variation. Knowing the variation exists, and even it's extent, doesn't seem terribly helpful.

Also, I'm not getting much evidence that "shelter-in-place" is related to reported cases of CV. Doesn't mean it's not, just that it's not visible here.

What would be nice is some data that provide information on personal activity. Google released some information on April 3, 2020 that may be just that. I've searched for things otherwise but haven't found anything.

## Appendix
Clearly, if you look at the first plot above, $R_0$ isn't constant. I believe there is a potential avenue to make a predictive model of the number of cases of CV using this observation. E.g., if $I(t)$ denotes the number of CV cases at time $t$, then $ I(t) = (R(t))^t. $ And just kinda throwing a wild guess out there, the plot above looks like a negative quadratic so we would/could have

$$ \log I(t) = -c (t - x_0)^2 + y_0. $$

where it looks like $x_0, y_0 > 0$ and $ 1 > c > 0 $. ($ x_0 > 0 $ haha). Thus,

$$ I(t) = e^{-c (t - x_0)^2 + y_0} $$

One could then basically combine $c$ and $e$ (I'm glossing over some algebra) to obtain


$$ I(t) = R_0^{(t - x_0)^2 + y_0} $$

I think $x_0$ represents something like the time that maximizes $I(t)$, $y_0$ something like how infectious the thing is (how great the extent of infection is), and $R_0$ is the parameter that represents the base rate of growth in infection somehow. It's also possible that a quadratic under-determines how this infection should be modeled. Long story ~~short~~ longer still: one could investigate:

$$ I(t) = R_0^{f(t)} $$

where $f(t)$ is concave-ish.